In [1]:
from getpass import getpass  
from elasticsearch import Elasticsearch, helpers

# Prompt the user to enter their Elastic Cloud ID and API Key securely
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Elastic API Key: ")

# Create an Elasticsearch client using the provided credentials
client = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,  # cloud id can be found under deployment management
    api_key=ELASTIC_API_KEY, # your username and password for connecting to elastic, found under Deplouments - Security
)

In [2]:
from datasets import load_dataset

ds = load_dataset("alpindale/two-million-bluesky-posts")

/Users/iulia/Documents/search options/search_options/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 2107530/2107530 [00:00<00:00, 2172877.37 examples/s]


In [38]:
ds["train"][0:1]

{'text': ["This is really interesting polling data about national public attitudes re: California.  It's from the LA Times, in January.  I wonder if this will change substantially in the next two years?  5233025.fs1.hubspotusercontent-na1.net/hubfs/523302..."],
 'created_at': ['2024-11-27T07:53:47.202Z'],
 'author': ['did:plc:5ug6fzthlj6yyvftj3alekpj'],
 'uri': ['at://did:plc:5ug6fzthlj6yyvftj3alekpj/app.bsky.feed.post/3lbw33zxvik24'],
 'has_images': [False],
 'reply_to': [None]}

We will only extract the relevant fields, and we will process them into the relevant datatype to match our desired mapping.

In [32]:
mappings = {
    "properties" : {
        "text" : {
            "type" : "keyword",
            "type" : "text"
        },
        "created_at": {
            "type": "date" 
        },
        "author" : {
            "type" : "keyword",
            "type" : "text"
        }
    }
}

In [ ]:
def first_element(document, key):
    if document[key]:
        text=document[key][0],
    else:
        text=None
    return text

In [26]:
# Define a function to convert DataFrame rows to Elasticsearch documents
def df_to_doc(df, index_name):
    for document in df:
        yield dict(_index=index_name, 
                   text=first_element(document, "text")
                   created_at=first_element(document,"created_at"),
                   author=first_element(document, "author"))

In [27]:
index_name = "bluesky"
# Create the Elasticsearch index with the specified name
client.indices.create(index=index_name, mappings=mappings)

# Use the Elasticsearch helpers.bulk() method to index the DataFrame data into Elasticsearch
load = helpers.bulk(client, df_to_doc(ds["train"], index_name))

IndexError: string index out of range

In [ ]:
client.inde